In [ ]:
import os
import requests
import base64
from io import BytesIO

from PIL import Image
import openai

def encode_image(image):
    buffer = BytesIO()
    image.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")
    
    
def get_current_synoptic_analysis(maps, map_urls):
    api_key = os.getenv("OPENAI_API_KEY")
    api_key="<key>"
    client = openai.OpenAI(api_key=api_key)
    prompt = f"""
        Please analyze these weather maps and provide a synoptic breakdown.
        Your audience is a meteorologist, so please be technical. Also be thorough.
        Please provide your response in markdown. The map urls are {", ".join(map_urls)}.
        Please include the urls in the markdown.
    """
    system = "You are a helpful meteorological assistant."

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": [{"type": "text", "text": prompt}]}
    ]

    for image in maps:
        base64_img = encode_image(image)
        messages[1]["content"].append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_img}"
            }
        })

    response = client.chat.completions.create(
        model = "gpt-4o",
        messages=messages
    )

    return response.choices[0].message.content


def download_files(date, hour):
    map_urls = [
        "https://climate.cod.edu/data/surface/US/contour/current/US.fronts.gif", # sfc
        f"https://climate.cod.edu/data/upper/US/contour/USvort.{date}.{hour}.gif", # 500mb vort
        f"https://climate.cod.edu/data/upper/US/700/US700rh.{date}.{hour}.gif", # 700mb rh
        f"https://climate.cod.edu/data/upper/US/850/US850dew.{date}.{hour}.gif", # 850 dpt
        f"https://climate.cod.edu/data/upper/US/contour/USthick.{date}.{hour}.gif" #1000-500mb del Z
    ]
    maps = []
    for url in map_urls:
        response = requests.get(url)
        image = Image.open(BytesIO(response.content))
        maps.append(image)

    return maps, map_urls

In [9]:
from datetime import datetime
date = datetime.now().strftime("%Y%m%d")
hour = 12
maps, map_urls = download_files(date, hour)
answer = get_current_synoptic_analysis(maps, map_urls)

In [11]:
from IPython.display import Markdown, display
display(Markdown(answer))

```markdown
## Synoptic Weather Analysis

### Surface Analysis
![Surface Analysis](https://climate.cod.edu/data/surface/US/contour/current/US.fronts.gif)

The surface map indicates a series of low-pressure systems across the central and eastern United States, with associated frontal boundaries. Notable features include:

- **Low Pressure over the Midwest:** Significant cyclogenesis, likely contributing to the southeastward flow of cold air.
- **Cold Fronts stretching to the Gulf Coast:** Marked by high-pressure ridging over the western U.S., contributing to clear, stable conditions.
- **Warm Sector:** Present over the southeastern U.S., promoting moist and buoyant air, leading to convective activity.

### 500 hPa Vorticity
![500mb Vorticity](https://climate.cod.edu/data/upper/US/contour/USvort.20250513.12.gif)

This map depicts a pronounced trough with high vorticity advection around the central Plains:

- **Vorticity Maximum:** Over the Rockies, indicative of a deepened upper-level trough, signaling increased instability.
- **Ridging over the West Coast:** Allows for enhanced downstream flow, contributing to energy transport into the trough.

### 700 hPa Relative Humidity and Temperature
![700mb RH](https://climate.cod.edu/data/upper/US/700/US700rh.20250513.12.gif)

At 700 hPa, humidity maxima and temperature fields highlight:

- **Moisture Convergence:** Over the southeastern states, correlating with active weather patterns.
- **Temperature Gradient:** Strong baroclinicity within the frontal zones.

### 850 hPa Dew Point
![850mb Dew Point](https://climate.cod.edu/data/upper/US/850/US850dew.20250513.12.gif)

Dew point values offer insights into low-level moisture:

- **High Dew Points:** Across the Gulf states, feeding into the convective systems to the north.
- **Moisture Advection:** Suggests that southerly flow is driving warm, moist air into collision zones.

### Thickness and Surface Pressure
![1000-500mb Thickness](https://climate.cod.edu/data/upper/US/contour/USthick.20250513.12.gif)

The thickness contours and surface isobars show:

- **Tight Thickness Gradient:** Corresponding with frontal systems and areas of cyclogenesis.
- **Wind Patterns:** Northeasterly flow influenced by high-pressure systems, particularly across the northeastern and northwestern regions.
- **Streamlines:** Illustrate upper-level dynamic support and convergence zones, enhancing the likelihood of storm development.

### Conclusion

Overall, the synoptic setup reveals a dynamic weather pattern with significant interaction between upper-level dynamics and surface features. The presence of strong vorticity and moisture convergence over the southeast supports continued convective activity, while the western high pressure provides stabilizing effects. The maps indicate potential for severe weather in the active regions through enhanced instability and atmospheric forcing.

--- 
*Links to data sources*:
- [Surface Analysis](https://climate.cod.edu/data/surface/US/contour/current/US.fronts.gif)
- [500 hPa Vorticity](https://climate.cod.edu/data/upper/US/contour/USvort.20250513.12.gif)
- [700 hPa RH](https://climate.cod.edu/data/upper/US/700/US700rh.20250513.12.gif)
- [850 hPa Dew Point](https://climate.cod.edu/data/upper/US/850/US850dew.20250513.12.gif)
- [1000-500mb Thickness](https://climate.cod.edu/data/upper/US/contour/USthick.20250513.12.gif)
```
